In [1]:
import re
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

import nltk
import gensim
import torch
from transformers import BertTokenizer
from wordcloud import STOPWORDS

warnings.filterwarnings("ignore")

# 1. Load dataset

In [2]:
df_train = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
df_test = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')
print('Training set shape = {}'.format(df_train.shape))
print('Test set shape = {}'.format(df_test.shape))

Training set shape = (7613, 5)
Test set shape = (3263, 4)


# 2 Data preprocessing
## 2.1 Text Cleaning
* convert text to lower case
* Remove the URLs
* Remove special characters that are attached to words, like tonight%Ûª
* Remove the hashtags (#)
* Remove the mentions (@)
* Separate punctations from words with space

In [3]:
# Text Cleaning

def clean(tweet):
    
    # Change to lower characters
    tweet = tweet.lower()
    
    # remove Urls
    tweet = re.sub(r"https?:\/\/t.co\/[A-Za-z0-9]+", "", tweet)
    
    # remove hashtags
    tweet = re.sub(r"#", "", tweet)
    
    # remove mentions
    tweet = re.sub(r"@", "", tweet)
    
    # Special characters
    tweet = re.sub(r"\x89û_", "", tweet)
    tweet = re.sub(r"\x89ûª", "", tweet)
    tweet = re.sub(r"\x89ûò", "", tweet)
    tweet = re.sub(r"\x89ûó", "", tweet)
    tweet = re.sub(r"\x89ûïWhen", "When", tweet)
    tweet = re.sub(r"\x89ûï", "", tweet)
    tweet = re.sub(r"China\x89ûªs", "China's", tweet)
    tweet = re.sub(r"let\x89ûªs", "let's", tweet)
    tweet = re.sub(r"\x89û÷", "", tweet)
    tweet = re.sub(r"\x89ûª", "", tweet)
    tweet = re.sub(r"\x89û\x9d", "", tweet)
    tweet = re.sub(r"å_", "", tweet)
    tweet = re.sub(r"\x89û¢", "", tweet)
    tweet = re.sub(r"\x89û¢åÊ", "", tweet)
    tweet = re.sub(r"fromåÊwounds", "from wounds", tweet)
    tweet = re.sub(r"åÊ", "", tweet)
    tweet = re.sub(r"åÈ", "", tweet)
    tweet = re.sub(r"JapÌ_n", "Japan", tweet)    
    tweet = re.sub(r"Ì©", "e", tweet)
    tweet = re.sub(r"å¨", "", tweet)
    tweet = re.sub(r"SuruÌ¤", "Suruc", tweet)
    tweet = re.sub(r"åÇ", "", tweet)
    tweet = re.sub(r"å£3million", "3 million", tweet)
    tweet = re.sub(r"åÀ", "", tweet)
        
    return tweet

In [4]:
df_train['text_cleaned'] = df_train['text'].apply(lambda s : clean(s))
df_test['text_cleaned'] = df_test['text'].apply(lambda s : clean(s))
print('The original: {}'.format(df_train.iloc[892]['text']))
print('The cleaned: {}'.format(df_train.iloc[892]['text_cleaned']))

The original: I can't bloody wait!! Sony Sets a Date For Stephen KingÛªs Û÷The Dark TowerÛª #stephenking #thedarktower http://t.co/J9LPdRXCDE  @bdisgusting
The cleaned: i can't bloody wait!! sony sets a date for stephen kings the dark tower stephenking thedarktower   bdisgusting


## 2.2 Word Tokenization
There are different methods for tokenizing text into words, such as:

1. TreebankWordTokenizer
2. WordPunctTokenizer
3. WhitespaceTokenizer
4. BERT Tokenizer (Here we use BERT)

### Tokenize the texts with BertTokemizer and encode them

In [5]:
from transformers import BertTokenizer

# Tokenize the sentences with BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Have a look at vocabulary
vocab = tokenizer.get_vocab()  # a dictionary
# print(vocab)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

### Prepare Labels

In [6]:
# get labels
labels = df_train['target'].tolist()
labels = torch.tensor(labels)
print('Labels shape = {}'.format(len(labels)))

Labels shape = 7613


### Convert sentences into token indexes

In [7]:
def sent2token(data, dataset_num):
    input_ids, attention_masks = [], []

    # 50 sentences for example, you can try more in your experiments
    for sent in data['text_cleaned'][:dataset_num]:
        encoded_dict = tokenizer.encode_plus(
                            ' '.join(sent.split()[:100]),    # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = 102,           # Pad & truncate all sentences. (control the length so that you can construct the input as matrix)
                            pad_to_max_length = True,       #(control the length so that you can construct the input as matrix)
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                       )
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
        
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    
    return input_ids, attention_masks


input_ids, attention_masks = sent2token(df_train, df_train.shape[0])
print('Cleaned: ', ' '.join(df_train['text_cleaned'][892].split()[:60]))
print('Token ids:', input_ids[892])
print('Attention_masks:', attention_masks[892])
print('The shape of input_ids = {}'.format(input_ids.shape))
print('The shape of attention_masks = {}'.format(attention_masks.shape))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Cleaned:  i can't bloody wait!! sony sets a date for stephen kings the dark tower stephenking thedarktower bdisgusting
Token ids: tensor([  101,  1045,  2064,  1005,  1056,  6703,  3524,   999,   999,  8412,
         4520,  1037,  3058,  2005,  4459,  5465,  1996,  2601,  3578,  4459,
         6834,  1996,  7662, 25509, 25114,  1038, 10521, 12349,  3436,   102,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0])
Attention_masks: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

### Split the dataset

In [8]:
from torch.utils.data import TensorDataset, random_split
dataset = TensorDataset(input_ids, attention_masks, labels)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset into training and validation by randomly selecting samples
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

6,090 training samples
1,523 validation samples


### Prepare the dataloder
Dataloader is a function of transformer that can combine several samples into one batch. This batch can be input to the first model to do the classification or training.

In [9]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
epochs = 4
batch_size = 4

# create the dataloader to load the data automatically in batches
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially (the original order).
            batch_size = batch_size # Evaluate with this batch size.
        )

### Prepare the model

In [10]:
from transformers import BertForSequenceClassification

#BertForSequenceClassification is used for multi-calss classification
#if you want to use a model for binary classification, you can use other BERT series models or construct a new model based on BertModel

cls_model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels.
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

params = list(cls_model.named_parameters())
print('The BERT model has {:} different named parameters.\n'.format(len(params)))
print('==== Embedding Layer ====\n')
for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print('\n==== First Transformer ====\n')
for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print('\n==== Output Layer ====\n')
for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

### Prepare the optimization

In [11]:
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

# set the optimizer
optimizer = AdamW(cls_model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

total_steps = len(train_dataloader) * epochs

# create the learning rate scheduler
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,  # you can change this value for your experiments
                                            num_training_steps = total_steps)

### Prepare functions to compute accuracy and elapsed time

In [12]:
import numpy as np
import time
import datetime

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

### Train the model!

In [13]:
import torch
print(torch.cuda.is_available())

True


In [14]:
import os
import random
from tqdm import tqdm
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# 判断是否使用GPU
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
if use_cuda:
    cls_model = cls_model.cuda()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    
    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. `train` just changes the *mode*, it doesn't *perform* the training.
    cls_model.train()
    
    # For each batch of training data...
    # step: train_input, batch: train_label
    for step, batch in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        
        # Progress update every 40 batches.
        if step % 5 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # update model
        cls_model.zero_grad()   
        outputs = cls_model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
        loss = outputs.loss    # used to get the gradients of the parameters and should optimize parameters
        logits = outputs.logits

        # Accumulate training loss over all of the batches, then calculate the average loss at the end.
        total_train_loss = total_train_loss + loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(cls_model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()   # imply the gradient and change the parameters

        # Update the learning rate for next step.
        scheduler.step()
        
    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.
    
    print("")
    print("Running Validation...")
    
    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    cls_model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0
    
    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        # As we unpack the batch, also copy each tensor to the GPU using `to` method.
        # `batch` contains three pytorch tensors:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.

            outputs = cls_model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask,
                            labels=b_labels
                           )
            loss, logits = outputs.loss, outputs.logits
            print(loss)
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.cpu().numpy()

        #logits: the probability of each class
        #multi-class predictions are obtained by taking the argmax of the logits
        #Notice: binary classification is different from multi-class classification when get the predictions

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 4 ========
Training...


  0%|          | 6/1523 [00:02<05:25,  4.66it/s]

  Batch     5  of  1,523.    Elapsed: 0:00:02.


  1%|          | 11/1523 [00:02<03:54,  6.45it/s]

  Batch    10  of  1,523.    Elapsed: 0:00:03.


...

  Batch 1,515  of  1,523.    Elapsed: 0:03:45.


100%|█████████▉| 1521/1523 [03:45<00:00,  6.78it/s]

  Batch 1,520  of  1,523.    Elapsed: 0:03:46.


100%|██████████| 1523/1523 [03:46<00:00,  6.73it/s]



  Average training loss: 0.23
  Training epcoh took: 0:03:46

Running Validation...
tensor(0.0010, device='cuda:0')
tensor(0.0027, device='cuda:0')
tensor(0.0040, device='cuda:0')
tensor(0.0015, device='cuda:0')
tensor(0.0072, device='cuda:0')
tensor(1.2946, device='cuda:0')
tensor(0.6978, device='cuda:0')
tensor(1.0814, device='cuda:0')
tensor(3.6203, device='cuda:0')
tensor(0.9694, device='cuda:0')
tensor(0.0577, device='cuda:0')
tensor(0.0068, device='cuda:0')
tensor(0.2618, device='cuda:0')
tensor(0.0048, device='cuda:0')
tensor(0.0841, device='cuda:0')
tensor(0.0171, device='cuda:0')
tensor(0.0014, device='cuda:0')
tensor(0.0176, device='cuda:0')
tensor(0.0043, device='cuda:0')
tensor(0.0009, device='cuda:0')
tensor(0.5886, device='cuda:0')
tensor(1.8988, device='cuda:0')
tensor(1.8909, device='cuda:0')
tensor(0.0017, device='cuda:0')
tensor(0.0097, device='cuda:0')
tensor(1.1597, device='cuda:0')
tensor(0.0009, device='cuda:0')
tensor(1.5852, device='cuda:0')
tensor(0.8754, devi

In [15]:
training_stats

[{'epoch': 1,
  'Training Loss': 0.5608938434408905,
  'Valid. Loss': 0.5182497981358857,
  'Valid. Accur.': 0.833989501312336,
  'Training Time': '0:03:48',
  'Validation Time': '0:00:13'},
 {'epoch': 2,
  'Training Loss': 0.47394096006982356,
  'Valid. Loss': 0.6111846990526466,
  'Valid. Accur.': 0.8353018372703412,
  'Training Time': '0:03:46',
  'Validation Time': '0:00:13'},
 {'epoch': 3,
  'Training Loss': 0.34363103201028844,
  'Valid. Loss': 0.7699520340647055,
  'Valid. Accur.': 0.8324584426946632,
  'Training Time': '0:03:46',
  'Validation Time': '0:00:13'},
 {'epoch': 4,
  'Training Loss': 0.23237613772256274,
  'Valid. Loss': 0.8543707726982678,
  'Valid. Accur.': 0.8346456692913385,
  'Training Time': '0:03:46',
  'Validation Time': '0:00:13'}]

In [16]:
outputs.loss

tensor(0.1465, device='cuda:0')

In [17]:
outputs.logits

tensor([[-0.3061, -0.9213],
        [ 2.5021, -2.9605],
        [ 2.7157, -3.0633]], device='cuda:0')

In [21]:
cls_model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

### Evaluate the model

In [22]:
def evaluate(model, test_data, batch_size):
    inputs_ids, attention_masks = sent2token(test_data, test_data.shape[0])
    test = TensorDataset(inputs_ids, attention_masks)
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=batch_size)
    
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda:
        model = model.cuda()

    predictions = []
    pred_labels = []
    t0 = time.time()
    count = 0
    
    model.eval()
    with torch.no_grad():
        for batch in test_dataloader:
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)

            outputs = model(b_input_ids, attention_mask=b_input_mask)
            loss, logits = outputs.loss, outputs.logits
            pred_label = np.argmax(outputs.logits.cpu(), axis=1)
            pred_labels.extend(pred_label.tolist())

            elapsed = format_time(time.time() - t0)
            print(' Batch {:>5,} of {:>5,}. Elapsed: {:}.'.format(count, len(test_dataloader), 
elapsed))
            count += 1

#                 # 获取模型输出的概率分布
#                 probabilities = torch.softmax(outputs.logits, dim=1)
#                 # 获取预测结果（这里选择概率最高的类别）
#                 _, predicted_labels = torch.max(probabilities, dim=1)
#                 predictions.extend(predicted_labels.tolist())
                      

    return pred_labels

In [23]:
pred_labels = evaluate(cls_model, df_test, batch_size)

 Batch     0 of   816. Elapsed: 0:00:00.
 Batch     1 of   816. Elapsed: 0:00:00.
 Batch     2 of   816. Elapsed: 0:00:00.
 Batch     3 of   816. Elapsed: 0:00:00.
 Batch     4 of   816. Elapsed: 0:00:00.
 Batch     5 of   816. Elapsed: 0:00:00.
 Batch     6 of   816. Elapsed: 0:00:00.
 Batch     7 of   816. Elapsed: 0:00:00.
 Batch     8 of   816. Elapsed: 0:00:00.
 Batch     9 of   816. Elapsed: 0:00:00.
 Batch    10 of   816. Elapsed: 0:00:00.
 Batch    11 of   816. Elapsed: 0:00:00.
 Batch    12 of   816. Elapsed: 0:00:01.
 Batch    13 of   816. Elapsed: 0:00:01.
 Batch    14 of   816. Elapsed: 0:00:01.
 Batch    15 of   816. Elapsed: 0:00:01.
 Batch    16 of   816. Elapsed: 0:00:01.
 Batch    17 of   816. Elapsed: 0:00:01.
 Batch    18 of   816. Elapsed: 0:00:01.
 Batch    19 of   816. Elapsed: 0:00:01.
 Batch    20 of   816. Elapsed: 0:00:01.
 Batch    21 of   816. Elapsed: 0:00:01.
 Batch    22 of   816. Elapsed: 0:00:01.
 Batch    23 of   816. Elapsed: 0:00:01.
 Batch    24 of 

In [24]:
pred_labels

[1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,


In [25]:
model_submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")
model_submission['target'] = np.round(pred_labels).astype('int')
model_submission.to_csv('model_submission.csv', index=False)
model_submission.describe()

,id,target
count,3263.000000,3263.000000
mean,5427.152927,0.406681
std,3146.427221,0.491290
min,0.000000,0.000000
25%,2683.000000,0.000000
50%,5500.000000,0.000000
75%,8176.000000,1.000000
max,10875.000000,1.000000


In [ ]:
masks_test.size()

In [ ]:
test = TensorDataset(ids_test, masks_test)
test[0][1]

In [ ]:
def plot_learning_curve(self):

    fig, axes = plt.subplots(nrows=K, ncols=2, figsize=(20, K * 6), dpi=100)

    for i in range(K):

        # Classification Report curve
        sns.lineplot(x=np.arange(1, self.epochs + 1), y=clf.models[i].history.history['val_accuracy'], ax=axes[i][0], label='val_accuracy')
        sns.lineplot(x=np.arange(1, self.epochs + 1), y=clf.scores[i]['validation']['precision'], ax=axes[i][0], label='val_precision')
        sns.lineplot(x=np.arange(1, self.epochs + 1), y=clf.scores[i]['validation']['recall'], ax=axes[i][0], label='val_recall')
        sns.lineplot(x=np.arange(1, self.epochs + 1), y=clf.scores[i]['validation']['f1'], ax=axes[i][0], label='val_f1')        

        axes[i][0].legend() 
        axes[i][0].set_title('Fold {} Validation Classification Report'.format(i), fontsize=14)

        # Loss curve
        sns.lineplot(x=np.arange(1, self.epochs + 1), y=clf.models[0].history.history['loss'], ax=axes[i][1], label='train_loss')
        sns.lineplot(x=np.arange(1, self.epochs + 1), y=clf.models[0].history.history['val_loss'], ax=axes[i][1], label='val_loss')

        axes[i][1].legend() 
        axes[i][1].set_title('Fold {} Train / Validation Loss'.format(i), fontsize=14)

        for j in range(2):
            axes[i][j].set_xlabel('Epoch', size=12)
            axes[i][j].tick_params(axis='x', labelsize=12)
            axes[i][j].tick_params(axis='y', labelsize=12)

    plt.show()

In [ ]:
test_dataloader = torch.utils.data.DataLoader(test, batch_size=batch_size)

In [ ]:
    for step, batch in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        
        # Progress update every 40 batches.
        if step % 5 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

In [ ]:
cls_model.plot_learning_curve()

# 保存学习曲线图像
plt.savefig('learning_curve.png')

# 显示学习曲线
plt.show()

In [ ]:
pd.DataFrame(training_stats).to_csv('/kaggle/working/training_stats-3.11-1.csv', index=False)

In [ ]:
y_pred = cls_model.predict(df_test)

model_submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")
model_submission['target'] = np.round(y_pred).astype('int')
model_submission.to_csv('model_submission.csv', index=False)
model_submission.describe()

### 4.2.2 Configuration & Model for one sentence

In [ ]:
from transformers import BertConfig, BertModel

# parameters for the model
configuration = BertConfig()
model = BertModel(configuration)
configuration = model.config
print(configuration)

In [ ]:
# load the pre-trained model 
# (you can also download the model from the link: https://huggingface.co/google-bert/bert-base-uncased, 
# and then load it from the local directory)
model = BertModel.from_pretrained('bert-base-uncased')
inputs = tokenizer("He remains characteristically confident and optimistic.",
                   return_tensors="pt")
print('Inputs:', inputs)

# the input is a dictionary with keys: input_ids, token_type_ids, attention_mask
outputs = model(**inputs)
last_hidden_states = outputs[0]
print('Last hidden states:', last_hidden_states)
print('Last hidden states size:',last_hidden_states.size())

### Reference
* Tutorial document
* tf + fine-tuned model: [Disaster NLP: Keras BERT using TFHub](https://www.kaggle.com/code/xhlulu/disaster-nlp-keras-bert-using-tfhub)
* tf + fine-tuned model, 预处理很好很多，比如有remove emoji，puncuations: [NLP - EDA, Bag of Words, TF IDF, GloVe, BERT](https://www.kaggle.com/code/vbmokin/nlp-eda-bag-of-words-tf-idf-glove-bert#Acknowledgements)
* pt + bert + 自定义结构，有交叉验证，代码类似，可参考结构整理代码: [BERT Baseline](https://www.kaggle.com/code/bibek777/bert-baseline)
* tf + 交叉验证 + ClassificationReport，超详细数据分析、数据清洗：[NLP with Disaster Tweets - EDA, Cleaning and BERT](https://www.kaggle.com/code/gunesevitan/nlp-with-disaster-tweets-eda-cleaning-and-bert/notebook#5.-Mislabeled-Samples)